In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json

In [2]:
def consulta_per_autor(ids):  
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")
    #options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana gráfica)

    s = Service(r"chromedriver-win32\chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=options) 

    # inicializando el navegador
    driver.get("https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros")
        
    try:
        element1 = (By.ID,'mat-input-1')
        element2 = (By.CSS_SELECTOR, "button.boton-buscar mdc-button mdc-button--raised mat-mdc-raised-button mat-accent mat-mdc-button-base".replace(" ","."))
        

        tarea1 = WebDriverWait(driver, 5)\
                    .until(EC.visibility_of_element_located(element1))
        tarea1.send_keys(str(ids))
        tarea2 = WebDriverWait(driver,5).until(EC.element_to_be_clickable(element2))
        tarea2.click()
        try:
            element3 = (By.XPATH, "/html/body/app-root/app-expel-listado-juicios/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section")
            tarea3 = WebDriverWait(driver,10).until(EC.element_to_be_clickable(element3)) #uso dependiendo de la velocidad de conexion
            texto_informacion = driver.find_element(By.XPATH,"/html/body/app-root/app-expel-listado-juicios/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section")
            texto_columnas = texto_informacion.text.split("\n")
            cons_no = int(len(texto_columnas)/5 -1)
            text_process = {}
                        
            for i in range(0,cons_no, 1):
                element4 = (By.XPATH,f"/html/body/app-root/app-expel-listado-juicios/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section/div[2]/div[{i+1}]/div[5]/a/mat-icon")
                tarea4 = WebDriverWait(driver,20).until(EC.element_to_be_clickable(element4))
                tarea4.click()
                element5 = (By.XPATH,f"/html/body/app-root/app-expel-listado-movimientos/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section/div/div[2]/div/div[2]/div/div[5]/a/mat-icon" )
                tarea5 = WebDriverWait(driver,20).until(EC.element_to_be_clickable(element5))
                tarea5.click()
                element6 = (By.XPATH,"/html/body/app-root/app-expel-listado-actuaciones/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section[2]/header/section[1]/section[2]/div/button[1]/span[2]")
                tarea6 = WebDriverWait(driver,20).until(EC.element_to_be_clickable(element6))
                tarea6.click()
                text_info = driver.find_element(By.XPATH, "/html/body/app-root/app-expel-listado-actuaciones/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/expel-informacion-busqueda/header")
                texto_columnas_info = text_info.text.split("\n")[6:] 
                text_process[f"Datos generales {i+1}"] = texto_columnas_info
                text_details = driver.find_element(By.XPATH, "/html/body/app-root/app-expel-listado-actuaciones/expel-sidenav/mat-sidenav-container/mat-sidenav-content/section/section[2]/mat-accordion")
                texto_columnas_details = text_details.text.split("\n")
                text_process[f"Detalles {i+1}"] = texto_columnas_details
                element7 = (By.CSS_SELECTOR,"button.botones btn-regresar mdc-button mat-mdc-button mat-primary mat-mdc-button-base".replace(" ",".")) 
                tarea7 =WebDriverWait(driver,30).until(EC.element_to_be_clickable(element7))
                tarea7.click()
                tarea8 = WebDriverWait(driver,30).until(EC.element_to_be_clickable(element7))
                tarea8.click()
                            
        except: 
            texto_informacion = f"El Id{ids} no tiene información"
            text_process= texto_informacion.split("\n")
            
    except Exception as e:    
        print(f"el ID: {ids} no tiene informacion")
    finally: 
        driver.quit()
    return ids, text_process

In [3]:
def paralelizar_consultas(ids, max_workers=10):
    resultados = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_id = {executor.submit(consulta_per_autor, id): id for id in ids}
        for future in as_completed(future_to_id):
            id = future_to_id[future]
            try:
                id, data = future.result()
                if data:
                    resultados.append((id, data))
                    print(f"consulta {id} realizada")
            except Exception as exc:
                print(f'Error en la consulta del ID {id}: {exc}')
    
    return resultados

In [7]:
consultas = ["0968599020001"]#, "0992339411001"]

In [57]:
resultados_= paralelizar_consultas(consultas)

consulta 0968599020001 realizada


In [40]:
def procesar_datos(data):
    resultados = []
    for id_value, valores in data:
        encabezados = ['No.', 'Fecha de ingreso', 'No. proceso', 'Acción /Infracción', 'Detalle']
        valores_procesados = valores[5:]  # Saltar los encabezados
        entradas = []

        for i in range(0, len(valores_procesados), len(encabezados)):
            entrada = dict(zip(encabezados, valores_procesados[i:i+len(encabezados)]))
            entradas.append(entrada)

        resultados.append({
            "ID": id_value,
            "Entradas": entradas
        })
    
    return resultados

In [58]:
def data_precessing(data):
    json_data = {}
    for i in range(0, len(data), 2):
        key = data[i].strip()
        value = data[i+1].strip() if i+1 < len(data) else ''
        json_data[key] = value

    return json_data    

In [59]:
def add_data (data):
    titulos = ['Número de proceso',
                'Fecha ingreso',
                'Materia',
                'Tipo de acción',
                'Delito/Asunto',
                'Judicatura',
                'Tipo de Ingreso',
                'No. proceso vinculado',
                'Actor/Ofendido:',
                'Demandado/Procesado:']
    for resultado in data:
        interations = int(len(resultado[1])/2)
        for j in range(interations):
            for i in range(len(titulos) - 1):
                if titulos[i] in resultado[1][f"Datos generales {j+1}"] and titulos[i+1] in resultado[1][f"Datos generales {j+1}"]:
                    index_i = resultado[1][f"Datos generales {j+1}"].index(titulos[i])
                    index_i1 = resultado[1][f"Datos generales {j+1}"].index(titulos[i+1])
                    if abs(index_i1 - index_i) < 2:
                        resultado[1][f"Datos generales {j+1}"].insert(index_i1, "")
                                
    return data


In [60]:
resultados_add = add_data(resultados_)

In [61]:
def data2json(data):

    for resultado in data:
        interations = int(len(resultado[1])/2)
        for i in range (interations): 
                resultado[1][f"Datos generales {i+1}"] = data_precessing(resultado[1][f"Datos generales {i+1}"])

    return data
    

In [62]:
resultados_finales = data2json(resultados_add)


In [89]:
def details2json(data): 
    records = []
    for i in range (0, len(data), 4): 
        record = {
            "Fecha": data[i],
            "Tipo de Documento": data[i+1],
            "Tipo de Archivo": data[i+2],
            "Descripción": data[i+3]
    }
        records.append(record)
    
    return records

In [105]:
resultados_finales[0][1]["Detalles 10"] 

['03/10/2023 09:54',
 'RECHAZAR REVOCATORIA Y/O REFORMA (RAZON DE NOTIFICACION)',
 'folder',
 'En Quito, martes tres de octubre del dos mil veinte y tres, a partir de las nueve horas y cincuenta y cuatro minutos, mediante boletas judiciales notifiqué el AUTO DE INTERLOCUTORIO que antecede a: EMPRESA ELECTRICA PUBLICA ESTRATEGICA CORPORACION NACIONAL DE ELECTRICIDAD CNEL EP en el correo electrónico luis.cando@cnel.gob.ec, alirio.cedeno@cnel.gob.ec. EMPRESA ELECTRICA PUBLICA ESTRATEGICA CORPORACION NACIONAL DE ELECTRICIDAD CNEL EP en el casillero No.1, No se notifica a: ALMAROSELEC S.A., PROCURADURIA GENERAL DEL ESTADO, por no haber señalado casillero electrónico. Certifico:',
 '',
 '',
 '',
 '',
 '',
 '',
 '02/10/2023 12:46',
 'RECHAZAR REVOCATORIA Y/O REFORMA (AUTO INTERLOCUTORIO)',
 'folder',
 'VISTOS: Agréguese al proceso el escrito que antecede.- En atención al mismo este tribunal analiza: ',
 'PRIMERO: La parte actora en su escrito de fecha 27 septiembre 2023 las 14:23 solicita la 

In [83]:
nn = [x for x in resultados_finales[0][1]["Detalles 4"] if x!='']

In [85]:
nn

['29/02/2024 10:43',
 'ACTA DE SORTEO',
 'folder',
 'Recibido en la ciudad de Manta el día de hoy, jueves 29 de febrero de 2024, a las 10:43, el proceso Tránsito COIP, Tipo de acción: Contravenciones de tránsito por Asunto: 389 contravenciones de tránsito de cuarta clase, inc.1, num. 6, seguido por: Empresa Electrica Publica Estrategica Corporacion Nacional de Electricidad Cnel Ep. Por sorteo de ley la competencia se radica en la UNIDAD JUDICIAL PENAL DE MANTA, conformado por Juez(a): Veliz Indacochea Zoila Janeth Que Reemplaza A Abg. Christian Luvin Quito Carpio. Secretaria(o): Almache Paredes Luis Javier Que Reemplaza A Acosta Vinces Carola. Proceso número: 13284-2024-06765 (1) Primera Instancia, con número de parte P17014487640210Al que se adjunta los siguientes documentos: 1) PETICIÓN INICIAL (ORIGINAL) Total de fojas: 3sr. JORGE RENATO HEREDIA GALLARDO Responsable de sorteo',
 '29/02/2024 10:43',
 'CARATULA DE JUICIO',
 'folder',
 'CARATULA']

IndexError: list index out of range

In [63]:
with open("prueba3.json", "w", encoding='utf8') as json_file:
    json.dump(resultados_finales, json_file, ensure_ascii=False, indent=4)

In [173]:
data_precessing(resultados[1][1]["Datos generales 5"])


{'Número de proceso': '09286202202415G',
 'Fecha ingreso': '06/05/2022 11:14',
 'Materia': 'CONTRAVENCIONES COIP',
 'Tipo de acción': 'DILIGENCIAS PREPROCESALES',
 'Delito/Asunto': 'SOLICITUD DE OFICIO',
 'Judicatura': 'UNIDAD JUDICIAL NORTE 2 PENAL CON SEDE EN EL CANTÓN GUAYAQUIL, PROVINCIA DEL GUAYAS',
 'Tipo de Ingreso': 'Ingreso directo',
 'No. proceso vinculado': '',
 'Actor/Ofendido:': 'Imveresa Importadora De Vehiculos Y Repuestos S.a.',
 'Demandado/Procesado:': ''}